In [208]:
# ==============================
# Standard Library Imports
# ==============================
from enum import Enum
import random

# ==============================
# Third-party Library Imports
# ==============================
import matplotlib.pyplot as plt
from IPython.display import display, Latex
from matplotlib.ticker import MultipleLocator
import numpy as np  # Original numpy
import scipy as sp
import pennylane as qml

# Pennylane numpy
import pennylane.numpy as pnp

In [209]:
# ==============================
# Setup for Quantum Computations
# ==============================
# PennyLane settings
coeffs = [-0.5]
obs = [qml.PauliZ(0)]
hamiltonian = qml.Hamiltonian(coeffs, obs)
dev = qml.device('default.mixed', wires=1)

In [210]:
class Data_Index(Enum):
    PHI = 0
    PARAS_START = 1
    THETA_X = 1
    PHI_Z = 2
    GAMMA_PS = 3
    TAU = 4

In [211]:
Data_set = pnp.array([
    [0.1, np.pi/2, np.pi/2],
    [0.2, np.pi/2, np.pi/2] 
])

In [212]:
Data_set[0]

tensor([0.1       , 1.57079633, 1.57079633], requires_grad=True)

In [213]:
def Dephase_factor(tau):
    """
    Calculate the dephasing factor (gamma) based on the provided tau value.

    Args:
        tau (float): Dephasing time constant.

    Returns:
        float: Calculated dephasing factor (gamma).
    """
    return 1 - np.exp(-2 * tau)

In [214]:
@qml.qnode(dev)
def circuit(paras):
    # Unpack parameters
    # gamma_dephase_in = Dephase_factor(paras[Data_Index.TAU.value])  
    phi = paras[Data_Index.PHI.value]
    theta_x = paras[Data_Index.THETA_X.value]
    phi_z = paras[Data_Index.PHI_Z.value]
    
    qml.RX(pnp.pi/2, wires = 0)
    
    qml.ApproxTimeEvolution(hamiltonian, phi, 1)
    # qml.PhaseDamping(gamma_dephase_in, wires = 0)

    qml.RZ(phi_z, wires = 0)  # phi_z
    
    qml.RX(theta_x, wires = 0)  # theta_x
    
    return qml.density_matrix(wires = 0)


@qml.qnode(dev)
def Post_selection_Dephase(paras):
    """ Take qnode from circuit_1 and calculate decoherence using kraus operator.
    
    Args:
        phi (float): Phi for Time-approximation. Pass by global variables:'Phi_global'

    Returns:
        qml.density_matrix: Density matrix of full qnode
    """
    # Unpack parameters 
    Gamma_ps = 0.8
    
    # Get density matrix from circuit_1
    density_matrix = circuit(paras)
    qml.QubitDensityMatrix(density_matrix, wires = 0)
    
    # Kraus operator for 2*2 matrix
    K = pnp.array([
        [pnp.sqrt(1 - Gamma_ps), 0],
        [0, 1]
    ])
    
    Numerator = K @ density_matrix @ K.conj().T
    Denominator = pnp.trace(Numerator)
    
    rho_ps = Numerator / Denominator
    
    qml.QubitDensityMatrix(rho_ps, wires = 0)
    
    return qml.density_matrix(wires = 0) 

# paras_plot = 1,1
# fig, ax = qml.draw_mpl(circuit)(paras_plot)

In [215]:
qml.qinfo.classical_fisher(Post_selection_Dephase)(Data_set[0])[0][0]

0.48838294029078466

In [216]:
def Cost_function(paras):
    """ Calculate Classical-Fisher-Information for qnode(=Post_selection_Dephase).
    
    Args:
        paras (Numpy array): [theta_init, tau_1, tau_2, tau_d1, tau_d2, tau_d3]

    Returns:
        _type_: CFI with minus(-) sign.
    """
      
    CFI = qml.qinfo.classical_fisher(Post_selection_Dephase)(paras[:])
    return -CFI[0][0]

In [217]:
print(Cost_function(Data_set[0]))

-0.48838294029078466


In [218]:
Data_set[0]

tensor([0.1       , 1.57079633, 1.57079633], requires_grad=True)

In [219]:
for i in range(len(Data_set[:])):
    print(Data_set[i])
    print(Cost_function(Data_set[i]))

[0.1        1.57079633 1.57079633]
-0.48838294029078466
[0.2        1.57079633 1.57079633]
-0.4332038638318201


In [224]:
test = np.array([1,2,3])

test[0:1]

array([1])